In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
movies = {
    'parasite' : 3,
    'suicide-squad' : 8,
    'birds-of-prey-and-the-fantabulous-emancipation-of-one-harley-quinn' : 3,
    'green-lantern' : 3,
    'transformers-revenge-of-the-fallen' : 8,
    'cats' : 2,
    'captain-marvel' : 21,
    'catwoman' : 1,
    'fifty-shades-of-grey' : 3,
    'about-time': 1,
    'frozen-2013': 4
}

In [3]:
movies = {
    'interstellar' : 8,
    'inception' : 11,
    'fantastic-four-2015' : 3,
    'logan-2017' : 4,
    'transformers-the-last-knight' : 2,
    'venom-2018' : 6,
    'transformers' : 6,
    'transformers-age-of-extinction' : 4,
    'transformers-dark-of-the-moon' : 4,
    'men-in-black-international' : 2,
    'joker' : 27,
    'titanic' : 2,
    'v-for-vendetta' : 4,
    'the-matrix' : 3,
    'the-dark-knight' : 15,
    'the-phantom-of-the-opera' : 0,
    'man-of-steel' : 11,
    'the-godfather' : 5,
    'justice-league' : 7,
    'whiplash' : 3,
    'la-la-land' : 3,
    'arrival' : 3,
    'blade-runner-2049' : 5,
    'gravity' : 8,
    'the-maze-runner' : 2
}

In [4]:
for movie_name in movies:
    totalPage = movies[movie_name]
    multi_page_result = [ ]
    for page in range( totalPage ):
        url = 'https://www.metacritic.com/movie/' + movie_name + '/user-reviews?page=' + str(page)

        html = requests.get( url, headers = { 'User-Agent' : 'Mozila/5.0' } )
        # Slow Down Error 뜨면 다시 get
        while ( str( html ) != '<Response [200]>' ):
            html = requests.get( url, headers = { 'User-Agent' : 'Mozila/5.0' } )
        soup = BeautifulSoup( html.text, 'html.parser' )

        # Get Reviews
        review_list = [ ]
        html_review_list = soup.findAll("div", attrs={"class":"review_body"})
        for div_line in html_review_list:
            span_list = div_line.find_all("span")
            for span_line in span_list:
                span_class = span_line.get("class")
                if not span_class:
                    text = span_line.get_text()
                    review_list.append(text.replace('\n\n', ''))
                elif len(span_class) > 1 and span_class[1]=='blurb_expanded':
                    text = span_line.get_text()
                    review_list.append(text.replace('\n\n', ''))

        # Get Label
        label_list = [ ]
        html_label_list = soup.select( 'div.metascore_w.user' )
        for line in html_label_list:
            label = line.attrs['class'][4]
            if label == 'negative':
                label_int = 0
            elif label == 'positive':
                label_int = 1
            elif label == 'mixed':
                label_int = 2
            label_list.append( label_int )
            
        for review, label in zip( review_list, label_list ):
            data = [ review, label ]
            multi_page_result.append( data )
            
        print( 'Success {} {}/{}'.format( movie_name, page + 1, totalPage ) )        
        
    review_df = pd.DataFrame( multi_page_result, columns = [ 'review', 'label' ] )
    
    review_df = review_df[review_df['label'] != 2] # Delete Mixed Review
    
    review_df.to_csv( movie_name + '_review.csv' )
    
    print( 'Crawling Success {}'.format( movie_name ) )

Success interstellar 1/8
Success interstellar 2/8
Success interstellar 3/8
Success interstellar 4/8
Success interstellar 5/8
Success interstellar 6/8
Success interstellar 7/8
Success interstellar 8/8
Crawling Success interstellar
Success inception 1/11
Success inception 2/11
Success inception 3/11
Success inception 4/11
Success inception 5/11
Success inception 6/11
Success inception 7/11
Success inception 8/11
Success inception 9/11
Success inception 10/11
Success inception 11/11
Crawling Success inception
Success fantastic-four-2015 1/3
Success fantastic-four-2015 2/3
Success fantastic-four-2015 3/3
Crawling Success fantastic-four-2015
Success logan-2017 1/4
Success logan-2017 2/4
Success logan-2017 3/4
Success logan-2017 4/4
Crawling Success logan-2017
Success transformers-the-last-knight 1/2
Success transformers-the-last-knight 2/2
Crawling Success transformers-the-last-knight
Success venom-2018 1/6
Success venom-2018 2/6
Success venom-2018 3/6
Success venom-2018 4/6
Success venom-2